# API Nakala

## Packages

In [19]:
using CSV
using DataFrames
using HTTP
using JSON
using Dates
using FileIO

ArgumentError: ArgumentError: Package FileIO not found in current path:
- Run `import Pkg; Pkg.add("FileIO")` to install the FileIO package.


## Identifiants

In [ ]:
path = @__DIR__
parentDir = dirname(path)
credentials = CSV.read(joinpath(parentDir,"credentials", "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "tnakala" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [ ]:
apitest = false

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

## Utilisateurs

### Lister les données

### Chercher une fichier précis dans une donnée

In [ ]:
title = "Z1J434"
lang = ""
firstImage = 1
lastImage = 2

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => lang
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
data = datas[1]
identifier = get(datas[1], "identifier", "")
files = get(data, "files", "")

In [ ]:
urls = Vector()
for i in firstImage:lastImage
  file = files[i]
  fileName = get(file, "name", "unknown")

  fileIdentifier = get(file, "sha1", "unknown")
  fileUrl = joinpath(apiurl, "data", identifier, fileIdentifier)

  img = Dict(
    "url" => fileUrl,
    "label" => fileName
  )

  push!(urls, img)
end

urls

In [ ]:
function splitImage(label, file, size)
  middle = size[2]/2
  l = floor(Int, middle+100)
  r = floor(Int, middle-100)
  leftPart = @view file[ :, 1:l]
  rightPart = @view file[ :, r:size[2]]
  save(string(splitext(label)[1], "-l", splitext(label)[2]), leftPart)
  save(string(splitext(label)[1], "-l", splitext(label)[2]), rightPart)
end

for img in urls
  label = get(img, "label", "unknown")
  file = download(get(img, "url", "")) |> load
  size = size(file)
  
  file = rot180(file)

  if size[1] < size[2]
    splitImage(label, file, size)
  else
    save(label, file)
  end

end